# Práctica Grupal: Diseño de Base de Datos

La imagen muestra el diseño de un modelo de base de datos relacional que contiene cuatro tablas principales: proveedores, piezas, suministro y categoría. A continuación, se explican los elementos clave de este modelo:

![Base de datos](imagen_1.jpg)

1. Tabla: Proveedores
Claves y relaciones:
codigoproveedor es la Primary Key de esta tabla, lo que significa que identifica de manera única a cada proveedor.
Es también Foreign Key en la tabla suministro, estableciendo una relación entre ambas tablas.

2. Tabla: Piezas
Claves y relaciones:
codigopieza es la Primary Key de esta tabla y también actúa como Foreign Key en la tabla suministro.
codigocategoria es una Foreign Key que establece la relación con la tabla categoría.

3. Tabla: Categoría
Claves y relaciones:
codigocategoria es la Primary Key de esta tabla, utilizada como Foreign Key en la tabla piezas.
Atributos:
nombre: Nombre de la categoría de piezas.

4. Tabla: Suministro
Claves y relaciones:
codigosuministro es la Primary Key de esta tabla.
Contiene las siguientes Foreign Keys:
codigoproveedor: Relacionada con la tabla proveedores.
codigopieza: Relacionada con la tabla piezas.

![Relaciones](imagen_2.jpg)


Relaciones clave:
La tabla suministro actúa como tabla intermedia (o de unión) entre proveedores y piezas, permitiendo registrar información sobre el suministro de piezas específicas por parte de ciertos proveedores.
La tabla piezas está vinculada con categoría, clasificando cada pieza dentro de una categoría específica.

# Código de Base de Datos

Este código demuestra cómo crear y gestionar una base de datos SQLite para un almacén, incluyendo la creación de tablas, la definición de relaciones entre ellas y la inserción de datos de ejemplo.

In [ ]:
#import pandas as pd
import sqlite3

# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("almacen.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
cursor= connection.cursor()

# Creación de Tablas

query_tabla_proveedores = """
CREATE TABLE Proveedores (
    CodigoProveedor INTEGER PRIMARY KEY,
    Nombre TEXT NOT NULL,
    Direccion TEXT,
    Ciudad TEXT,
    Provincia TEXT
);
"""
cursor.execute(query_tabla_proveedores)

query_tabla_categoria = """
CREATE TABLE Categoria (
    CodigoCategoria INT PRIMARY KEY,
    Nombre TEXT NOT NULL
);
"""
cursor.execute(query_tabla_categoria)


query_tabla_pieza="""
CREATE TABLE Piezas (
    CodigoPieza INT PRIMARY KEY,
    Nombre TEXT NOT NULL,
    Color TEXT,
    Precio FLOAT NOT NULL,
    CodigoCategoria INT,
    FOREIGN KEY (CodigoCategoria) REFERENCES Categoria(CodigoCategoria)
        ON DELETE SET NULL
        ON UPDATE CASCADE
);
"""

cursor.execute(query_tabla_pieza)

query_tabla_suministro="""
CREATE TABLE Suministro (
    CodigoSuministro INT PRIMARY KEY,
    CodigoProveedor INT NOT NULL,
    CodigoPieza INT NOT NULL,
    Cantidad INT NOT NULL,
    Fecha DATE NOT NULL,
    FOREIGN KEY (CodigoProveedor) REFERENCES Proveedores(CodigoProveedor)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
    FOREIGN KEY (CodigoPieza) REFERENCES Piezas(CodigoPieza)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);
"""
cursor.execute(query_tabla_suministro)

# Relleno de ejemplos para las tablas
datos_categoria = [
    (1, 'Resistencias'),
    (2, 'Capacitores'),
    (3, 'Semiconductores'),
    (4, 'Conectores')
]

datos_proveedores = [
    (1, 'ElectroTech S.L.', 'Av. Innovación 123', 'Madrid', 'Madrid'),
    (2, 'Componentes Globales', 'Calle Tecnología 456', 'Barcelona', 'Cataluña'),
    (3, 'Distribuidora del Norte', 'Av. Central 789', 'Bilbao', 'País Vasco'),
    (4, 'Electronix S.A.', 'Calle Comercio 101', 'Sevilla', 'Andalucía'),
    (5, 'Conexiones Avanzadas', 'Av. Industria 202', 'Valencia', 'Comunidad Valenciana')
]

datos_piezas = [
    (1, 'Resistencia 10kΩ', 'Marrón', 0.05, 1),
    (2, 'Capacitor 100uF', 'Azul', 0.10, 2),
    (3, 'Transistor NPN', 'Negro', 0.50, 3),
    (4, 'Diodo LED', 'Rojo', 0.15, 3),
    (5, 'Conector USB Tipo-C', 'Plateado', 1.00, 4)
]

datos_suministro = [
    (1, 1, 1, 1000, '2024-01-01'),
    (2, 1, 2, 500, '2024-01-02'),
    (3, 2, 3, 200, '2024-01-03'),
    (4, 3, 4, 750, '2024-01-04'),
    (5, 4, 5, 300, '2024-01-05')
]

# Inserción de datos
cursor.executemany("INSERT INTO Categoria (CodigoCategoria, Nombre) VALUES (?, ?)", datos_categoria)
cursor.executemany("INSERT INTO Proveedores (CodigoProveedor, Nombre, Direccion, Ciudad, Provincia) VALUES (?, ?, ?, ?, ?)", datos_proveedores)
cursor.executemany("INSERT INTO Piezas (CodigoPieza, Nombre, Color, Precio, CodigoCategoria) VALUES (?, ?, ?, ?, ?)", datos_piezas)
cursor.executemany("INSERT INTO Suministro (CodigoSuministro, CodigoProveedor, CodigoPieza, Cantidad, Fecha) VALUES (?, ?, ?, ?, ?)", datos_suministro)

# Confirmar cambios y cerrar conexión
connection.commit()
connection.close()

# Código de Consulta

Este código utiliza SQLite y Pandas para consultar una base de datos y convertir los resultados en un DataFrame de Pandas. Esto facilita trabajar con los datos en formato tabular para análisis y manipulación.

In [10]:
import sqlite3
import pandas as pd

# Conectamos con la base de datos almacen.db
connection = sqlite3.connect("almacen.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
cursor = connection.cursor()

def sql_query(query):
    # Ejecuta la query
    cursor.execute(query) 

    # Almacena los datos de la query 
    ans = cursor.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans, columns=names)

# Definimos la consulta
query = "SELECT * FROM Piezas"

# Ejecutamos y almacenamos los resultados en un DataFrame
df = sql_query(query)
print(df)


   CodigoPieza               Nombre     Color  Precio  CodigoCategoria
0            1     Resistencia 10kΩ    Marrón    0.05                1
1            2      Capacitor 100uF      Azul    0.10                2
2            3       Transistor NPN     Negro    0.50                3
3            4            Diodo LED      Rojo    0.15                3
4            5  Conector USB Tipo-C  Plateado    1.00                4


In [7]:
connection.close()

In [19]:
import sqlite3
import pandas as pd

# Conexión a la base de datos almacen.db
connection = sqlite3.connect("almacen.db")
cursor = connection.cursor()

# Función para ejecutar consultas y devolver un DataFrame
def sql_query(query):
    cursor.execute(query)  # Ejecuta la consulta
    ans = cursor.fetchall()  # Recupera los datos de la consulta
    names = [description[0] for description in cursor.description]  # Obtiene los nombres de las columnas
    return pd.DataFrame(ans, columns=names)

# Solicitar al usuario el nombre de la tabla
tabla = input("¿Qué tabla deseas consultar? ")

# Crear la consulta SQL dinámicamente
query = f"SELECT * FROM {tabla}"

# Ejecutar la consulta y mostrar los resultados
try:
    df = sql_query(query)
    print(f"Resultados de la tabla '{tabla}':")
    print(df)
except Exception as e:
    print(f"Error al realizar la consulta: {e}")

# Cerrar la conexión a la base de datos
connection.close()


¿Qué tabla deseas consultar?  Piezas


Resultados de la tabla 'Piezas':
   CodigoPieza               Nombre     Color  Precio  CodigoCategoria
0            1     Resistencia 10kΩ    Marrón    0.05                1
1            2      Capacitor 100uF      Azul    0.10                2
2            3       Transistor NPN     Negro    0.50                3
3            4            Diodo LED      Rojo    0.15                3
4            5  Conector USB Tipo-C  Plateado    1.00                4
